# Szyfrowanie uwierzytelnione z danymi uzupełniającymi (AEAD) (ang. Authenticated Encryption with Additional Data)

W czasie transmisji pakietowych część danych musi pozostać otwarta aby np. umożliwić właściwy routing pakietu. Jednocześnie chcemy, aby dane te nie zostały zmienione w czasie transmisji. Z kolei pole danych (tzw. payload) ma być utajnione. Jednocześnie chcemy zapewnić zablokować możliwość modyfikacji danych otwartych i zaszyfrowanych. W tym celu opracowano tryby pracy szyfrów które łączą te dwa, pozornie odrębne zadania w jeden prymityw kryptograficzny. Dalej zostaną zaprezentowane przykłady ochrony danych w tym trybie.

## AKA

Przed wykonaniem szyfrowania i deszyfrowania strony muszą być wyposażone w klucz tajny. Jeżeli współdzielona tajemnica jest tylko hasłem o niskiej entropii to należy przeprowadzić procedurę *key streching*. Oprócz współdzielonego hasła jej parametrem jest współdzielona i jawna liczba losowa *salt*. Od strony teoretycznej procedury te zostaną omówione przy innej okazji (patrz rozdział KDF). Tutaj zostanie przedstawiona tylko procedura która realizuje to zadanie. Procedurę tę muszą wykonać zarówno nadawca jak i odbiorca.

In [2]:
import binascii
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
# Key stretching
password = b'password123'
salt = get_random_bytes(16)
secret_key = PBKDF2(password, salt, 32, count=100000, hmac_hash_module=SHA256)
print("Salt= ", binascii.hexlify(salt))
print("Secret Key= ", binascii.hexlify(secret_key))

Salt=  b'fa9d10bcff59158f169628f2f534cd11'
Secret Key=  b'aaa8ea02475256974b24f476bf6a0bad810bf621a52a55b444702a89b222fa82'


## AES CCM (CTR with CBC MAC)

Nadawca i odbiorca współdzielą klucz (np. wyprowadzony z tego samego hasła jak w WPA-PSK). Procedurę uzgadniania klucza nazywa się AKA (Authentication and Key Agreement) przy czym uwierzytelnienie dotyczy tutaj użytkowników.

### Szyfrowanie

Jednorazową liczbę losową 'nonce' służącą do wyprowadzenia klucza unikalnego dla każdej wiadomości przesyła się w postaci jawnej.

In [3]:
import os, binascii
from Crypto.Cipher import AES
# Prepare cipher
message = 'To jest text do zaszyfrowania'
cipher = AES.new(secret_key, AES.MODE_CCM)
anonce = cipher.nonce 
payload = message.encode('utf-8')
#cipher.update(anonce)
ciphertext, mac = cipher.encrypt_and_digest(payload)
# Send encrypted data
print("Recipient receives:")
print("Nonce= ", binascii.hexlify(anonce),len(anonce))
print("Ciphertext= ", binascii.hexlify(ciphertext))
print("MAC= ", binascii.hexlify(mac),len(mac))
message = anonce + ciphertext + mac

Recipient receives:
Nonce=  b'a655229de803de1c1ecbe0' 11
Ciphertext=  b'c2567833ef92fa1aef3b18f0bc7329fdee64483aca1c53e8c9dd70e3dc'
MAC=  b'5b1ca124c8a958738d3114a6253d30d4' 16


### Deszyfrowanie

Po odebraniu pakietu należy sprawdzić, czy suma kontrolna dla nagłówka i szyfrogramu jest poprawna, a następnie odszyfrować payload

In [5]:
anonce = message[:11]
mac = message[-16:]
ciphertext = message[11:-16]
decryptor = AES.new(secret_key, AES.MODE_CCM, nonce=anonce)
try:
    #decryptor.update(anonce)
    payload = decryptor.decrypt_and_verify(ciphertext, mac)
    print("Message= ", payload.decode('utf-8'))
except:
    print("Incorrect decryption")

Message=  To jest text do zaszyfrowania


## ChaCha20 i Poly1305

Schemat użycia trybu z innym zestawem algorytmów jest dokładnie taki sam. Na przykład szyfrowanie w trybie AEAD z wykorzystaniem szyfru ChaCha20 i funkcji MAC Poly1305 wygląda następująco (należy pamiętać, że tym razem klucz współdzielony **musi** mieć 256 bitów (32 bajty)):

### Szyfrowanie

In [ ]:
import os, binascii
from Crypto.Cipher import ChaCha20_Poly1305
# Prepare data to encrypt
confidential_payload = 'To jest text do zaszyfrowania'.encode('utf-8')
additional_open_data = b'12345678'
# Prepare cipher
encryptor = ChaCha20_Poly1305.new(key=secret_key)
encryptor.update(additional_open_data)
ciphertext, mac = encryptor.encrypt_and_digest(confidential_payload)
message = encryptor.nonce + additional_open_data + ciphertext + mac
# Send encrypted data
print("Recipient receives:")
print("Nonce= ", binascii.hexlify(encryptor.nonce), len(encryptor.nonce))
print("Header= ", binascii.hexlify(additional_open_data))
print("Ciphertext= ", binascii.hexlify(ciphertext))
print("MAC= ", binascii.hexlify(mac), len(mac))

### Deszyfrowanie

Deszyfrowanie przebiega wg teo samego schematu: sprawdź sumę kontrolną a potem odszyfruj treść komunikatu

In [ ]:
anonce = message[:12]
header = message[12:(12+8)]
ciphertext = message[(12+8):-16]
mac = message[-16:]
decryptor = ChaCha20_Poly1305.new(key=secret_key, nonce=anonce)
try:
    decryptor.update(header)
    payload = decryptor.decrypt_and_verify(ciphertext, mac)
    print("Message= ", payload.decode('utf-8'))
except:
    print("Incorrect decryption")